In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import json

# File paths
base_path = "/kaggle/input/legal-qa-indian-law-multi"

# Load JSON files
constitution_df = pd.read_json(f"{base_path}/constitution_qa.json")
crpc_df = pd.read_json(f"{base_path}/crpc_qa.json")
ipc_df = pd.read_json(f"{base_path}/ipc_qa.json")

# Load JSONL file
synthetic_df = pd.read_json(f"{base_path}/synthetic_legal_qa_dataset_unique_questions.jsonl", lines=True)

# Combine all
df = pd.concat([constitution_df, crpc_df, ipc_df, synthetic_df], ignore_index=True)

# Check preview
df.sample(5)


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,question,answer,id,source,metadata,Answer
16531,What is the burden of proof in a criminal case...,"Beyond reasonable doubt, rests with the prosec...",NaN,synthetic_rule_llm,"{'topic': 'IPC', 'question_type': 'procedural'...",NaN
12754,What does 'gaining wrongfully' and 'losing wro...,A person is said to gain wrongfully when such ...,NaN,NaN,NaN,NaN
3958,When was the Orissa Land Reforms (Second Amend...,1975,NaN,NaN,NaN,NaN
4855,Which section pertains to the effect of such w...,Section 422,NaN,NaN,NaN,NaN
7673,What is the time limit for the complainant to ...,The complainant may appeal the decision within...,NaN,NaN,NaN,NaN


In [2]:
!pip install -q sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.2 MB/s eta 0:00:00:00:0100:01


In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm

# Use Sentence-BERT (MiniLM)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Ensure no NaN questions
df = df[df['question'].notnull()].reset_index(drop=True)

# Create embeddings
questions = df['question'].tolist()
print(f"Embedding {len(questions)} questions...")

question_embeddings = model.encode(questions, show_progress_bar=True)

# Store embeddings as numpy array
question_embeddings = np.array(question_embeddings)


2025-07-27 07:35:05.945986: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753601706.225328      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753601706.310066      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding 16918 questions...


Batches:   0%|          | 0/529 [00:00<?, ?it/s]

In [4]:
import faiss
import pickle

# Step 1: Initialize FAISS index
embedding_dim = question_embeddings.shape[1]  # should be 384 for MiniLM
index = faiss.IndexFlatL2(embedding_dim)

# Step 2: Add question embeddings to index
index.add(question_embeddings)
print("Total vectors in FAISS index:", index.ntotal)

# Step 3: Save FAISS index and question-answer map
faiss.write_index(index, "legal_qa_faiss.index")

# Save mapping of questions and answers
qa_mapping = df[['question', 'answer']].to_dict(orient="records")
with open("qa_mapping.pkl", "wb") as f:
    pickle.dump(qa_mapping, f)

# Copy to /kaggle/working is already default


ModuleNotFoundError: No module named 'faiss'

In [5]:
!pip install -q faiss-cpu


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 30.1 MB/s eta 0:00:0000:0100:01m


In [6]:
import faiss
import pickle

# Get vector size (should be 384 for MiniLM)
embedding_dim = question_embeddings.shape[1]

# Initialize FAISS index
index = faiss.IndexFlatL2(embedding_dim)

# Add vectors to index
index.add(question_embeddings)

print("Total vectors in FAISS index:", index.ntotal)

# Save index to disk
faiss.write_index(index, "legal_qa_faiss.index")

# Save Q&A mapping for retrieval
qa_mapping = df[['question', 'answer']].to_dict(orient="records")
with open("qa_mapping.pkl", "wb") as f:
    pickle.dump(qa_mapping, f)


Total vectors in FAISS index: 16918


In [7]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pickle

# Load FAISS index
index = faiss.read_index("legal_qa_faiss.index")

# Load QA mapping
with open("qa_mapping.pkl", "rb") as f:
    qa_mapping = pickle.load(f)

# Load the same model used for encoding
model = SentenceTransformer('all-MiniLM-L6-v2')

def get_answer(query, top_k=3):
    # Encode the query
    query_embedding = model.encode([query])
    
    # Search similar vectors
    distances, indices = index.search(np.array(query_embedding), top_k)
    
    # Retrieve top-k results
    results = []
    for idx in indices[0]:
        qa = qa_mapping[idx]
        results.append(qa)
    
    return results

# 🔍 Try a test query:
query = "What is the punishment for theft under IPC?"
results = get_answer(query)

# 📜 Display
for i, res in enumerate(results):
    print(f"\nResult {i+1}:")
    print(f"Q: {res['question']}")
    print(f"A: {res['answer']}")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Result 1:
Q: What is the punishment for theft under Section 379 of the IPC?
A: Imprisonment of either description for a term which may extend to three years, or with fine, or with both.

Result 2:
Q: What is the punishment for kidnapping under IPC?
A: The punishment varies depending on the circumstances, including the age of the victim and the intent.

Result 3:
Q: What is the punishment if the offence is theft?
A: Imprisonment for 10 years and fine.


In [1]:
!pip install -q unsloth


In [3]:
!pip install -q --no-cache-dir unsloth accelerate peft bitsandbytes transformers datasets wandb


In [ ]:
import os
os.kill(os.getpid(), 9)


In [2]:
import unsloth
print("Unsloth version:", unsloth.__version__)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-07-27 08:06:47.531469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753603607.889691     131 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753603607.998170     131 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth version: 2025.7.8


In [8]:
!pip install -q unsloth peft accelerate bitsandbytes transformers


In [9]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

# Load the distilled DeepSeek 8B model via Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = 2048,
    dtype = "bfloat16",        # Use "float16" if bfloat16 is unsupported on your GPU
    load_in_4bit = True,
)


==((====))==  Unsloth 2025.7.8: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device does not support bfloat16. Will change to float16.


In [10]:
import json

# Path to your JSONL file
file_path = "/kaggle/input/legal-qa-indian-law-multi/synthetic_legal_qa_dataset_unique_questions.jsonl"

# Load first 5 samples to inspect the structure
examples = []
with open(file_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 5:
            break
        examples.append(json.loads(line.strip()))

# Display
for idx, ex in enumerate(examples):
    print(f"\n--- Example {idx+1} ---")
    for k, v in ex.items():
        print(f"{k}: {v}")



--- Example 1 ---
question: Explain the rights granted by Section 12 of the Indian Penal Code.
answer: This section ensures legal aid for individuals against sexual harassment.
source: synthetic_rule_llm
metadata: {'topic': 'IPC', 'question_type': 'definition', 'difficulty': 'medium', 'jurisdiction': 'India'}

--- Example 2 ---
question: What is the punishment for defamation under Indian Penal Code?
answer: According to Code of Criminal Procedure, Section 144 states that a court must issue a warrant first.
source: synthetic_rule_llm
metadata: {'topic': 'IPC', 'question_type': 'definition', 'difficulty': 'medium', 'jurisdiction': 'India'}

--- Example 3 ---
question: Who can file a complaint under Indian Penal Code regarding domestic violence?
answer: The punishment for murder under Indian Penal Code is typically fine up to ₹1,000.
source: synthetic_rule_llm
metadata: {'topic': 'IPC', 'question_type': 'procedure', 'difficulty': 'medium', 'jurisdiction': 'India'}

--- Example 4 ---
ques

In [12]:
import json

file_path = "/kaggle/input/legal-qa-indian-law-multi/synthetic_legal_qa_dataset_unique_questions.jsonl"

with open(file_path, "r") as f:
    for i, line in enumerate(f):
        sample = json.loads(line)
        if "answer" not in sample:
            print(f"Missing 'answer' at line {i + 1}: {sample}")


Missing 'answer' at line 585: {'question': "How does the Consumer Rights Act 2015 define 'satisfactory quality' in relation to goods?", 'Answer': 'Durable, safe, and functional as a reasonable person would expect, considering its description, price, and other factors.', 'source': 'synthetic_rule_llm', 'metadata': {'topic': 'Consumer Rights', 'question_type': 'definition', 'difficulty': 'medium', 'jurisdiction': 'UK'}}
Missing 'answer' at line 1488: {'question': "Define the term 'wrongful restraint' as per the IPC.", 'Answer': 'Unlawfully preventing someone from proceeding to any place where they have a right to go.', 'source': 'synthetic_rule_llm', 'metadata': {'topic': 'IPC', 'question_type': 'definition', 'difficulty': 'easy', 'jurisdiction': 'India'}}
Missing 'answer' at line 1493: {'question': "How does the IPC define 'fraud'?", 'Answer': 'Wrongfully deceiving another person causing monetary loss or other harm.', 'source': 'synthetic_rule_llm', 'metadata': {'topic': 'IPC', 'questio

In [14]:
dataset = []
with open(file_path, "r") as f:
    for line in f:
        sample = json.loads(line)
        if "question" in sample and "answer" in sample:
            dataset.append({
                "instruction": sample["question"],
                "input": "",
                "output": sample["answer"]
            })


In [20]:
from datasets import Dataset

# Convert list to HuggingFace Dataset
train_dataset = Dataset.from_list(dataset)


In [21]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = 2048,
    load_in_4bit = True,  # Efficient
    dtype = None,         # Will auto-select float16 for your GPU
)


==((====))==  Unsloth 2025.7.8: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [22]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)


Unsloth 2025.7.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [26]:
def formatting_func(example):
    return f"""### Instruction:
{example["instruction"]}

### Input:
{example["input"]}

### Response:
{example["output"]}"""


In [27]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "output",  # this gets ignored anyway when using formatting_func
    max_seq_length = 2048,
    packing = False,
    formatting_func = formatting_func,  # ✅ use the fixed formatting
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 50,
        learning_rate = 2e-4,
        fp16 = True,
        logging_steps = 1,
        output_dir = "outputs",
        optim = "paged_adamw_8bit",
        save_steps = 25,
        save_total_limit = 2,
    ),
)


ValueError: Unsloth: The `formatting_func` should return a list of processed strings.